<a href="https://colab.research.google.com/github/ArulrajGopal/Spark-Guide/blob/main/GogCol/07_schema_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

from pyspark.sql import SparkSession

#create spark session
spark= SparkSession.builder.appName('mysparksession').getOrCreate()

#create spark context
sc = spark.sparkContext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=9383654f07cc2648fe28f3b8878f19d2cdd5adc469aee535863b9510d1f9d983
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
!ls

circuits.csv  sample_data


In [ ]:
circuits_df = spark.read\
    .format("csv")\
    .option('Header',True)\
    .option('InferSchema',True)\
    .load('circuits.csv')

In [ ]:
circuits_df.printSchema()

root
 |-- circuitId: integer (nullable = true)
 |-- circuitRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
circuits_df = spark.read\
    .format("csv")\
    .option('Header',True)\
    .load('circuits.csv')

In [ ]:
circuits_df.printSchema()

root
 |-- circuitId: string (nullable = true)
 |-- circuitRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- alt: string (nullable = true)
 |-- url: string (nullable = true)



In [ ]:
schema = {
'circuitId': 'INT',
 'circuitRef':'STRING',
 'name':'STRING',
 'location':'STRING',
 'country':'STRING',
 'lat':'Double',
 'lng':'Double',
 'alt':'int',
 'url':'STRING'
}

column_list = []
for column, datatype in schema.items():
    column_list.append(f"cast({column} as {datatype}) {column}")
    data_type_converted_df = circuits_df.selectExpr(column_list)

In [ ]:

data_type_converted_df.printSchema()

root
 |-- circuitId: integer (nullable = true)
 |-- circuitRef: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- country: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- url: string (nullable = true)



In [3]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [4]:
dataDF = [('james','1991-04-01','M',3000),
  ('Michel','2000-05-19','M',4000),
  ('Robert','1978-09-05','M',4000),
  ('Maria','1967-12-01','F',4000),
  ('Jen','1980-02-17','F',-1)
]


schema = StructType([StructField('name', StringType(), True),
         StructField('dob', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])


df = spark.createDataFrame(dataDF, schema)
df.printSchema()
df.show(truncate=False)

root
 |-- name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+------+----------+------+------+
|name  |dob       |gender|salary|
+------+----------+------+------+
|james |1991-04-01|M     |3000  |
|Michel|2000-05-19|M     |4000  |
|Robert|1978-09-05|M     |4000  |
|Maria |1967-12-01|F     |4000  |
|Jen   |1980-02-17|F     |-1    |
+------+----------+------+------+



In [5]:
dataDF = [(('James','','Smith'),'1991-04-01','M',3000),
  (('Michael','Rose',''),'2000-05-19','M',4000),
  (('Robert','','Williams'),'1978-09-05','M',4000),
  (('Maria','Anne','Jones'),'1967-12-01','F',4000),
  (('Jen','Mary','Brown'),'1980-02-17','F',-1)
]

structureSchema = StructType([
        StructField('name', StructType([
             StructField('firstname', StringType(), True),
             StructField('middlename', StringType(), True),
             StructField('lastname', StringType(), True)
             ])),
         StructField('id', StringType(), True),
         StructField('gender', StringType(), True),
         StructField('salary', IntegerType(), True)
         ])

df2 = spark.createDataFrame(dataDF,structureSchema)
df2.printSchema()
df2.show(truncate=False)

root
 |-- name: struct (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- middlename: string (nullable = true)
 |    |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: integer (nullable = true)

+--------------------+----------+------+------+
|name                |id        |gender|salary|
+--------------------+----------+------+------+
|{James, , Smith}    |1991-04-01|M     |3000  |
|{Michael, Rose, }   |2000-05-19|M     |4000  |
|{Robert, , Williams}|1978-09-05|M     |4000  |
|{Maria, Anne, Jones}|1967-12-01|F     |4000  |
|{Jen, Mary, Brown}  |1980-02-17|F     |-1    |
+--------------------+----------+------+------+

